Download dataset used in this notebook with cURL:

```console
#!/bin/bash
curl -L -o ~/Downloads/google-play-store-apps.zip\
  https://www.kaggle.com/api/v1/datasets/download/lava18/google-play-store-apps
```

# Google play store apps analysis
Data downloaded from https://www.kaggle.com/datasets/lava18/google-play-store-apps

The dataset contains details of the applications on Google Play. There are 10 features that describe the given app:
- App
- Rating
- Category
- Reviews
- Size
- Installs
- Type
- Price
- Content Rating
- Genres
- Last updated
- Current Version
- Android Version

## Extract

In [1]:
# imports
import pandas as pd

In [25]:
# read apps data into memory
apps = pd.read_csv("./google-play-store-apps/googleplaystore.csv")
apps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [26]:
# read reviews data into memory
reviews = pd.read_csv("./google-play-store-apps/googleplaystore_user_reviews.csv")
reviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [27]:
# function for basic info
def show_info(df: pd.DataFrame, varName: str | None=None ) -> None:
    """
    Shows basic information of the pd.DataFrame

    Input:
        df - pandas.DataFrame

    Output:
        printed statements on:
        - shape
        - columns names
    """

    # some basic info on the datasets
    if(varName): print(f"\nInfo about {varName}:")
    print(f"Number of rows and cols:{df.shape}")
    print(f"Column names: {list(df.columns)}")
    print(f"The columns in the dataframe has the following types:")
    print(df.dtypes)    


In [28]:
# show info
show_info(apps, "apps")


Info about apps:
Number of rows and cols:(10841, 13)
Column names: ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
The columns in the dataframe has the following types:
App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


In [29]:
# show info for reviews
show_info(reviews, "reviews")


Info about reviews:
Number of rows and cols:(64295, 5)
Column names: ['App', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity', 'Sentiment_Subjectivity']
The columns in the dataframe has the following types:
App                        object
Translated_Review          object
Sentiment                  object
Sentiment_Polarity        float64
Sentiment_Subjectivity    float64
dtype: object


In [30]:
# function for extraction
def extract(file_path: str) -> pd.DataFrame:

    """ 
    Extracting data from csv and read into a pandas.DataFrame

    Input:
        file_path: str - path to csv that is to be read into memory

    Output:
        data: pandas.DataFrame - dataframe containing the data
    """

    #read csv
    data=pd.read_csv(file_path)
    print("data read from csv DONE")

    #print info
    show_info(data)

    return data

### EDA


In [31]:
# count nans
apps.isna().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [32]:
reviews.isna().sum()

App                           0
Translated_Review         26868
Sentiment                 26863
Sentiment_Polarity        26863
Sentiment_Subjectivity    26863
dtype: int64

## Transform


I am going to further study apps and reviews in the category: "FOOD_AND_DRINK".



In [33]:
# new dataframe with apps of "FOOD_AND_DRINK" category
apps_food = apps[apps.Category == "FOOD_AND_DRINK"]
apps_food.head(2)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
1176,McDonald's,FOOD_AND_DRINK,3.6,145323,42M,"10,000,000+",Free,0,Everyone,Food & Drink,"July 31, 2018",5.15.0,4.4 and up
1177,Easy and Fast Recipes,FOOD_AND_DRINK,NaN,95,12M,"50,000+",Free,0,Everyone,Food & Drink,"January 6, 2018",3.0.1,2.3 and up


In [34]:
apps_food.shape

(127, 13)

In [35]:
apps_food.isna().sum()

App                0
Category           0
Rating            18
Reviews            0
Size               0
Installs           0
Type               0
Price              0
Content Rating     0
Genres             0
Last Updated       0
Current Ver        0
Android Ver        0
dtype: int64

In [43]:
# define a tranform function
def transform(app, review, category, min_rating, min_reviews):
    """
    Transforms the two dataframes by joining the two and selecting the relevant columns

    Input:

    Output:
    """

    #print statements for observability
    print(f"Transforming data to curate a dataset with all {category} apps and their "
          f"corresponding reviews with a rating of at least {min_rating} and "
          f"{min_reviews} reviews \n")
    
    #drop duplicates
    review = review.drop_duplicates()
    app = app.drop_duplicates(subset=["App"])

    #filter dataframes to include only the category given as input
    subset_apps= app.loc[app.Category==category, :]
    subset_reviews = review.loc[review.App.isin(subset_apps.App), ["App", "Sentiment_Polarity"] ]

    # aggerate the mean sentiment polarity
    aggregated_reviews = subset_reviews.groupby(by="App").mean()

    #join it back to subset_reviews df
    joined_apps_reviews = subset_apps.join(aggregated_reviews, on="App", how="left")

    #Keep only needed columns
    filtered_apps_reviews = joined_apps_reviews.loc[:, ["App", "Rating", "Reviews", "Installs", "Sentiment_Polarity"]]

    #convert reviews, keep only values with an avg rating of >= min_rating stars and >= min_reviews reviews
    filtered_apps_reviews = filtered_apps_reviews.astype({"Reviews": "int32"})
    top_apps = filtered_apps_reviews.loc[(filtered_apps_reviews.Rating >= min_rating) & (filtered_apps_reviews.Reviews > min_reviews), :]

    # sorting top apps, replace nans with 0, reset index (drop,inplace)
    top_apps.sort_values(by=["Rating", "Reviews"], ascending=False, inplace=True)
    top_apps.reset_index(drop=True, inplace=True)

    # persistt this df as top_apps.csv file
    top_apps.to_csv("top_apps.csv")

    print(f"The transformed dataframe, which includes {top_apps.shape[0]} rows "
          f"and {top_apps.shape[1]} columns has been persisted, and will be returned")
    
    return top_apps

top_apps_data = transform(apps, reviews, "FOOD_AND_DRINK", 4.0, 1000)
top_apps_data.head()

Transforming data to curate a dataset with all FOOD_AND_DRINK apps and their corresponding reviews with a rating of at least 4.0 and 1000 reviews 

The transformed dataframe, which includes 57 rows and 5 columns has been persisted, and will be returned


/var/folders/4d/x1s6d4b54_585bp8jb_sh1yh0000gn/T/ipykernel_36098/1253274092.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_apps.sort_values(by=["Rating", "Reviews"], ascending=False, inplace=True)


,App,Rating,Reviews,Installs,Sentiment_Polarity
0,SarashpazPapion (Cooking with Chef Bowls),4.8,1250,"50,000+",NaN
1,Domino's Pizza USA,4.7,1032935,"10,000,000+",0.226971
2,Tastely,4.7,611136,"10,000,000+",NaN
3,Delicious Recipes,4.7,129737,"1,000,000+",NaN
4,BeyondMenu Food Delivery,4.7,51517,"1,000,000+",0.408743


## Loading Data
Loading the transformed data int o a SQL database with pandas and sqlite3

In [42]:
import sqlite3

In [45]:
def load(dataframe, database_name, table_name):

    #create connection object
    con = sqlite3.connect(database_name)

    # write the data to the spesifiec table
    dataframe.to_sql(name=table_name, con=con, if_exists="replace", index=False)
    print("Original df has been loaded to sqlite")

    # read the data, and return the result
    loaded_df = pd.read_sql(sql=f"SELECT * FROM {table_name}", con=con)
    print("the loaded df had been read from sqlite for validation")

    try:
        assert dataframe.shape == loaded_df.shape
        print(f"Succes! The data in the {table_name} table have successfullt been loaded and validated")
    except AssertionError:
        print("Dataframe shape is not consistent before and after loadning. Take a close look!")

load(top_apps_data,"market_research","top_apps")

Original df has been loaded to sqlite
the loaded df had been read from sqlite for validation
Succes! The data in the top_apps table have successfullt been loaded and validated


# Running the pipeline
Running the pipeline end-to-end


In [46]:
# imports
import pandas as pd
import sqlite3

In [47]:
#extract data
final_apps_data = extract("./google-play-store-apps/googleplaystore.csv")
final_reviews_data = extract("./google-play-store-apps/googleplaystore_user_reviews.csv")

data read from csv DONE
Number of rows and cols:(10841, 13)
Column names: ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
The columns in the dataframe has the following types:
App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object
data read from csv DONE
Number of rows and cols:(64295, 5)
Column names: ['App', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity', 'Sentiment_Subjectivity']
The columns in the dataframe has the following types:
App                        object
Translated_Review          object
Sentiment                  object
Sentiment_Polarity        float64
Sentiment_Subje

In [49]:
# transform data
top_apps_data = transform(final_apps_data, final_reviews_data, "FOOD_AND_DRINK", 4.0, 1000)

Transforming data to curate a dataset with all FOOD_AND_DRINK apps and their corresponding reviews with a rating of at least 4.0 and 1000 reviews 

The transformed dataframe, which includes 57 rows and 5 columns has been persisted, and will be returned


/var/folders/4d/x1s6d4b54_585bp8jb_sh1yh0000gn/T/ipykernel_36098/1253274092.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_apps.sort_values(by=["Rating", "Reviews"], ascending=False, inplace=True)


In [50]:
load(top_apps_data,"market_research","top_apps")

Original df has been loaded to sqlite
the loaded df had been read from sqlite for validation
Succes! The data in the top_apps table have successfullt been loaded and validated
